In [2]:
from tsai.models import InceptionTime, TST, ResNet
# from tsai.data.core import get_UCR_data
# from tsai.learner import TSClassifier
import os
import numpy as np
import sys
print(sys.executable)
import sys
sys.path.insert(1, '../src/')
from CNN_classifier_model import CNNClassifier, train_model, evaluate_model
from config import raw_data_path, univariate_data_path, processed_data_path
from preprocessing_modules import create_time_windows_with_labels
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import roc_auc_score, average_precision_score
import matplotlib.pyplot as plt
from collections import Counter
import torch.nn as nn
import torch
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from sklearn.utils.class_weight import compute_class_weight
import wandb
wandb.login(key="5f15eb7efc1e0e939ccc83345338a0b8c24e2fbc")


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


/home/nwertheim/miniconda3/bin/python


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/nwertheim/.netrc
wandb: Currently logged in as: noorwertheim to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [3]:
'''MY DATA'''
train_dir = os.path.join(univariate_data_path, "target_univariate_no_PCA_train.npy")
train_data = np.load(train_dir, allow_pickle=True)
# train_data = pd.DataFrame(train_data)
test_dir = os.path.join(univariate_data_path, "target_univariate_no_PCA_test.npy")
test_data = np.load(test_dir, allow_pickle=True)
# test_data = pd.DataFrame(test_data)
print(len(train_data))
print(len(test_data))
print(train_data[0])
from torch.utils.data import DataLoader, TensorDataset, random_split
from sklearn.utils.class_weight import compute_class_weight

from collections import Counter

# Example dataset with windows and labels
train_windows, train_labels = create_time_windows_with_labels(train_data)
test_windows, test_labels = create_time_windows_with_labels(test_data)

# Count label distribution
train_label_counts = Counter(train_labels)
test_label_counts = Counter(test_labels)

# Print the counts
print("Train label distribution:")
print(f"  Term (0): {train_label_counts[0]}")
print(f"  Preterm (1): {train_label_counts[1]}")

print("\nTest label distribution:")
print(f"  Term (0): {test_label_counts[0]}")
print(f"  Preterm (1): {test_label_counts[1]}")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# Convert to tensors
train_windows_tensor = torch.tensor(train_windows, dtype=torch.float32)
train_labels_tensor = torch.tensor(train_labels, dtype=torch.float32)
test_windows_tensor = torch.tensor(test_windows, dtype=torch.float32)
test_labels_tensor = torch.tensor(test_labels, dtype=torch.float32)

train_windows_tensor = train_windows_tensor.unsqueeze(1)  # (1071, 1, 12000)
test_windows_tensor = test_windows_tensor.unsqueeze(1)    # (899, 1, 12000)

train_labels_tensor = train_labels_tensor.long()
test_labels_tensor = test_labels_tensor.long()

744
192
{'record_name': 'Hopper-2024_09_17_06_51_03-0000010009-0001-chan0', 'signal': array([[ -5.57103828],
       [-11.39992725],
       [ -9.53483776],
       ...,
       [  0.73235611],
       [ -0.1656677 ],
       [  1.35035514]]), 'fs': 20, 'preterm': 0}
Train label distribution:
  Term (0): 3360
  Preterm (1): 5748

Test label distribution:
  Term (0): 390
  Preterm (1): 2322


In [4]:
import torch
import optuna
import numpy as np
from tsai.models.ResNet import ResNet
from fastai.learner import Learner
from fastai.data.core import DataLoaders
from fastai.losses import CrossEntropyLossFlat
from sklearn.model_selection import KFold
from torch.utils.data import TensorDataset, Subset
from sklearn.metrics import average_precision_score
from fastai.metrics import Metric
from fastai.callback.wandb import WandbCallback

# Create TensorDataset from your data
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

dataset = TensorDataset(train_windows_tensor, train_labels_tensor)
print(len(train_windows_tensor))
# Custom Average Precision Metric
class AveragePrecision(Metric):
    def __init__(self):
        self.pred = []
        self.target = []

    def reset(self):
        self.pred, self.target = [], []

    def accumulate(self, learn):
        preds = learn.pred.detach().softmax(dim=-1)[:, 1]  # Get probabilities for class 1
        targs = learn.y.detach()
        self.pred.append(preds.cpu())
        self.target.append(targs.cpu())

    @property
    def value(self):
        preds = torch.cat(self.pred).numpy()
        targs = torch.cat(self.target).numpy()
        return average_precision_score(targs, preds)

    @property
    def name(self): return "avg_precision"

def objective(trial):
    print('Trial:', trial)
    print(f"Starting trial {trial.number}")

    learning_rate = trial.suggest_float("lr", 1e-6, 1e-3, log=True)
    batch_size = trial.suggest_categorical("batch_size", [16, 32, 64])

    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    avg_precisions = []

    for train_idx, val_idx in kf.split(np.arange(len(dataset))):
        train_dataset = Subset(dataset, train_idx)
        val_dataset = Subset(dataset, val_idx)

        dls = DataLoaders.from_dsets(
            train_dataset,
            val_dataset,
            bs=batch_size,
            shuffle=True,
            num_workers=0
        )

        model = ResNet(c_in=1, c_out=2).to(device)

        learn = Learner(
            dls, model,
            loss_func=CrossEntropyLossFlat(),
            metrics=AveragePrecision()
        )

        learn.fit_one_cycle(5, lr_max=learning_rate)
        ap = learn.validate()[1]  
        avg_precisions.append(ap)
        

    return np.mean(avg_precisions)

# Run Optuna study
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=25)

print("Best trial:")
print(study.best_trial)

# Save the study
import joblib
joblib.dump(study, "resnet_optuna_study.pkl")


[I 2025-05-07 10:56:19,503] A new study created in memory with name: no-name-a4b02e67-3095-4902-aaea-99b111c4b8a7


9108
Trial: <optuna.trial._trial.Trial object at 0x150563676e40>
Starting trial 0


epoch,train_loss,valid_loss,avg_precision,time
0,0.655383,0.653454,0.741731,00:12
1,0.651264,0.646102,0.715427,00:12
2,0.645258,0.639191,0.714635,00:12
3,0.635654,0.631454,0.716686,00:12
4,0.627002,0.625457,0.720744,00:12


epoch,train_loss,valid_loss,avg_precision,time
0,0.653243,0.650743,0.719794,00:12
1,0.653369,0.658620,0.699558,00:12
2,0.642702,0.645745,0.700780,00:12
3,0.620058,0.633236,0.726879,00:12
4,0.634980,0.641035,0.701313,00:12


epoch,train_loss,valid_loss,avg_precision,time
0,0.661305,0.655038,0.699396,00:12
1,0.656545,0.661755,0.698952,00:12
2,0.652398,0.660880,0.661323,00:12
3,0.646407,0.651921,0.711266,00:12
4,0.639621,0.652648,0.703179,00:12


...

epoch,train_loss,valid_loss,avg_precision,time
0,0.654434,0.658571,0.680413,00:12
1,0.656220,0.652223,0.699257,00:11
2,0.649324,0.651868,0.694430,00:12
3,0.649075,0.662185,0.684807,00:11
4,0.643081,0.670114,0.705305,00:11


epoch,train_loss,valid_loss,avg_precision,time
0,0.657012,0.688424,0.635826,00:11
1,0.651293,0.661235,0.695277,00:11
2,0.647872,0.675983,0.629912,00:11
3,0.644960,0.651862,0.667424,00:11
4,0.641741,0.643799,0.703639,00:11


[I 2025-05-07 11:06:39,355] Trial 1 finished with value: 0.7211595271016022 and parameters: {'lr': 0.00041002518836429395, 'batch_size': 64}. Best is trial 1 with value: 0.7211595271016022.


Trial: <optuna.trial._trial.Trial object at 0x1505635786b0>
Starting trial 2


epoch,train_loss,valid_loss,avg_precision,time
0,0.672321,0.657413,0.706260,00:12
1,0.654836,0.646791,0.734847,00:12
2,0.646915,0.645454,0.710844,00:12
3,0.658023,0.644606,0.722293,00:12


In [12]:
# import torch
# import optuna
# import numpy as np
# from tsai.models.ResNet import ResNet
# from fastai.learner import Learner
# from fastai.data.core import DataLoaders
# from fastai.metrics import accuracy
# from sklearn.model_selection import KFold
# from torch.utils.data import TensorDataset, Subset
# from fastai.callback.wandb import WandbCallback
# from fastai.data.core import DataLoaders
# from fastai.learner import Learner
# from fastai.metrics import accuracy
# from fastai.losses import CrossEntropyLossFlat
# from sklearn.model_selection import KFold
# from torch.utils.data import Subset
# from tsai.models.ResNet import ResNet

# # Create TensorDataset from your data
# dataset = TensorDataset(train_windows_tensor, train_labels_tensor)

# def objective(trial):
#     print('Trial: ', trial)
#     learning_rate = trial.suggest_float("lr", 1e-6, 1e-3, log=True)
#     batch_size = trial.suggest_categorical("batch_size", [16, 32, 64])

#     kf = KFold(n_splits=5, shuffle=True, random_state=42)
#     accs = []

#     for train_idx, val_idx in kf.split(np.arange(len(dataset))):
#         train_dataset = Subset(dataset, train_idx)
#         val_dataset = Subset(dataset, val_idx)

#         dls = DataLoaders.from_dsets(
#             train_dataset,
#             val_dataset,
#             bs=batch_size,
#             shuffle=True,
#             num_workers=0  # for compatibility and reproducibility
#         )
#         model = ResNet(c_in=1, c_out=2)
#         learn = Learner(dls, model, loss_func=CrossEntropyLossFlat(), metrics=accuracy)

#         learn.fit_one_cycle(5, lr_max=learning_rate)
#         acc = learn.validate()[1].item()
#         accs.append(acc)

#     return np.mean(accs)

# study = optuna.create_study(direction="maximize")
# study.optimize(objective, n_trials=25)

# print("Best trial:")
# print(study.best_trial)

# # Save the study if you want
# import joblib
# joblib.dump(study, "resnet_optuna_study.pkl")


[I 2025-05-06 16:29:27,414] A new study created in memory with name: no-name-549a4ffa-a502-4302-868a-480a67ab348b


Trial:  <optuna.trial._trial.Trial object at 0x14ac5879e810>


epoch,train_loss,valid_loss,accuracy,time


[W 2025-05-06 16:35:49,338] Trial 0 failed with parameters: {'lr': 0.0005133692498451997, 'batch_size': 16} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/home/nwertheim/miniconda3/lib/python3.12/site-packages/optuna/study/_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "/scratch-local/nwertheim.11597988/ipykernel_1556808/2838686348.py", line 44, in objective
    learn.fit_one_cycle(5, lr_max=learning_rate)
  File "/home/nwertheim/miniconda3/lib/python3.12/site-packages/fastai/callback/schedule.py", line 121, in fit_one_cycle
    self.fit(n_epoch, cbs=ParamScheduler(scheds)+L(cbs), reset_opt=reset_opt, wd=wd, start_epoch=start_epoch)
  File "/home/nwertheim/miniconda3/lib/python3.12/site-packages/fastai/learner.py", line 272, in fit
    self._with_events(self._do_fit, 'fit', CancelFitException, self._end_cleanup)
  File "/home/nwertheim/miniconda3/lib/python3.12/site-pack

KeyboardInterrupt: 